In [ ]:
from sklearn.datasets import make_classification

# Generate fake classification data
n_samples = 1000  # Number of samples
n_features = 10  # Number of features
n_classes = 4    # Number of classes

X, y = make_classification(
    n_samples=n_samples,
    n_features=n_features,
    n_informative=n_features,
    n_redundant=0,
    n_classes=n_classes,
    random_state=42
)

In [ ]:
X_label,y_label = X_train[:400],y_train_true[:400]
X_unlabelled = X_train[400:]

In [ ]:
import pandas as pd
import numpy as np

#sklearn.datasets generate np.arrays. Converting to pandas 
X_label = pd.DataFrame(X_label)
y_label = pd.Series(y_label)
X_unlabelled = pd.DataFrame(X_unlabelled)

# Self Learning Algorithm
while True:
    model = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=n_classes,
        random_state=42)

    # Train the model
    model.fit(X_label, y_label)

    # Get probability predictions on unlabeled data
    X_unlabelled.reset_index(drop=True, inplace=True)
    y_pred = model.predict_proba(X_unlabelled)
    
    # get samples where probability >0.9 for atleast one class
    index = [ index for index,x in enumerate(np.max(y_pred,axis=1)) if x > 0.90]
    if len(index)==0:
        break
        
    temp = X_unlabelled.iloc[index]
    
    # drop high probability samples from unlabeled data and append to labeled data
    X_unlabelled.drop(index,inplace=True)
    
    pred = pd.Series(model.predict(temp))
    
    X_label=X_label.append(temp,ignore_index=True)
    y_label=y_label.append(pred,ignore_index=True)

In [ ]:
model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=n_classes,
    random_state=42
)

# Train the model
model.fit(X_label, y_label)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test_true, y_pred)
print("Accuracy:", accuracy)